In [ ]:
!pip install -U sentence-transformers

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

import pandas as pd

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
qa_model = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [ ]:
df = pd.read_excel('Multiprocessors.xlsx')
dataset = df.values

In [ ]:
embeddingsOfTheSubTopics = model.encode(dataset[:, 2])

In [ ]:
question = 'does cache listen to the snoop bus ?'
ecodedQuestion = model.encode(question)

In [ ]:
cos_similarities = util.cos_sim(embeddingsOfTheSubTopics, ecodedQuestion)

In [ ]:
cos_similarities

tensor([[0.1148],
        [0.2105],
        [0.3830],
        [0.3939],
        [0.6472],
        [0.3616],
        [0.1709],
        [0.2324],
        [0.3177],
        [0.1792],
        [0.1740]])

In [ ]:
def getTheRelevantRow(question):
  max = cos_similarities[0]
  maxIndex = 0
  for idx, score in enumerate(cos_similarities):
    if (score > max):
      max = score
      maxIndex = idx

  return max, maxIndex

In [ ]:
selectedSubTopicRow = getTheRelevantRow(question)[1]
selectedFileName = dataset[selectedSubTopicRow][3]
print(selectedFileName)

Multiprocessors_Bus_snooping


In [ ]:
def getRelevantPassage(filename):
    with open('Files/' + filename + '.txt', encoding="utf8") as r:
        lines = r.readlines()
        return lines

In [ ]:
selectedPassage = getRelevantPassage(selectedFileName)[0]
selectedPassage

'\ufeffDedicated bus for coherency control. At every write access, cache controllers share addresses through the snoop bus. Each cache listens to the snoop bus,to check “if other caches are updating blocks that I also have cached”. Write-invalidate protocols:invalidate own copyNext access to same block is a miss!. Write-through caching: memory has up-to-date copy. Write-back caching: snoop in other caches to find most recent copy(more complex snooping, more snoop traffic, less memory traffic). Write-update protocols:update own copy (a.k.a. write-broadcast)Next access to same block is a hitNeed to share data+addressin snoop bus for every write(Increased snoop bus lines more power, less cache misses)'

In [ ]:
def answerForTheQuestion(question):
  print(question)
  print(qa_model(question=question, context=selectedPassage))

In [ ]:
answerForTheQuestion(question)

does cache listen to the snoop bus ?
{'score': 0.0663423165678978, 'start': 118, 'end': 153, 'answer': 'Each cache listens to the snoop bus'}
